# Credit Risk Evaluator

In [42]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


## Retrieve the Data

The data is located in the Challenge Files Folder:

* `lending_data.csv`

Import the data using Pandas. Display the resulting dataframe to confirm the import was successful.

In [16]:
# Import the data
credit_score_df = pd.read_csv('Resources/lending_data.csv')
credit_score_df.head()


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [17]:
# Check data types and unique rows
print(credit_score_df.dtypes)
print(len(credit_score_df.index))

loan_size           float64
interest_rate       float64
borrower_income       int64
debt_to_income      float64
num_of_accounts       int64
derogatory_marks      int64
total_debt            int64
loan_status           int64
dtype: object
77536


In [15]:
#drop duplicates
# credit_score_df.drop_duplicates(inplace=True)
# print(len(credit_score_df.index))

5229


## Predict Model Performance

You will be creating and comparing two models on this data: a Logistic Regression, and a Random Forests Classifier. Before you create, fit, and score the models, make a prediction as to which model you think will perform better. You do not need to be correct! 

Write down your prediction in the designated cells in your Jupyter Notebook, and provide justification for your educated guess.

* Logistic regression models apply a function that returns a probability of a value of 0 or 1. A random forest creates
several small decision trees, then combines the predictions. Logistic regression becomes less accurate if there are too many variables with little explanatory power. The model often overfits the training data and poorly predicts the test data. I predict the logistic regression model will prove more accurate for this exercise. I'm basing this prediction on research conducted by Kirasich, Kaitlin, et al (2018) [that studied the acuracy of logistic regression versus random forest modeling](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://scholar.smu.edu/cgi/viewcontent.cgi?article=1041&context=datasciencereview#:~:text=In%20general%2C%20logistic%20regression%20performs,variables%20increases%20in%20a%20dataset.). Their conclusion was that that logistic regression performs
better when the number of noise variables is less than or equal to the number of explanatory variables. Looking at the columns in the dataframe, it appears that all are likely to have explanatory power, although loan size and total debt may only be relevant in the context of total income. Even if debt-to-income explains more of the loan status than loan size or total debt, that is still less than half the variables.<br>


 ###### Kirasich, Kaitlin, Smith, Trace, & Sadler, Bivin Ph.D (2018). Random Forest vs Logistic Regression: Binary Classification for Heterogeneous Datasets. *SMU Data Science Review, 1(3)*, 22. 


## Split the Data into Training and Testing Sets

In [20]:
# Split the data into X_train, X_test, y_train, y_test
X = credit_score_df.drop('loan_status', axis=1)
y = credit_score_df['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Create, Fit and Compare Models

Create a Logistic Regression model, fit it to the data, and print the model's score. Do the same for a Random Forest Classifier. You may choose any starting hyperparameters you like. 

Which model performed better? How does that compare to your prediction? Write down your results and thoughts in the designated markdown cell.

In [25]:
# Train a Logistic Regression model and print the model score
#  Scale data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [44]:
# Fit regression
log_classifier = LogisticRegression().fit(X_train_scaled, y_train)
print(f"Training Data Score: {log_classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {log_classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.9942908240473243
Testing Data Score: 0.9936545604622369


In [38]:
# Confusion matrix
y_true = y_test
y_pred = log_classifier.predict(X_test_scaled)
confusion_matrix(y_true, y_pred)

array([[18652,   113],
       [   10,   609]], dtype=int64)

In [39]:
# Accuracy
TP = 609
FP = 113
TN = 18652
FN = 10
accuracy = (TP + TN) / (TP + FP + TN + FN)
print(accuracy)

0.9936545604622369


In [41]:
#  Classification
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.98      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



In [43]:
# Train a Random Forest Classifier model and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

Training Score: 0.9975409272252029
Testing Score: 0.9917457697069748


In [49]:
# Confusion matrix
y_true_f = y_test
y_pred_f = clf.predict(X_test_scaled)
confusion_matrix(y_true_f, y_pred_f)

array([[18665,   100],
       [   60,   559]], dtype=int64)

In [47]:
# Accuracy
TP = 559
FP = 100
TN = 18665
FN = 68
accuracy = (TP + TN) / (TP + FP + TN + FN)
print(accuracy)

0.9913366336633663


In [50]:
#  Classification
print(classification_report(y_true_f, y_pred_f))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.90      0.87       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



As predicted, the logistic regression model was more accurate. The accuracy difference was small at .994 versus .992. Looking at the confusion matrix, the largest difference appears to be that the forest tree model had a higher rate of false negatives. This is also reflected in the classification report that showed a lower sensitivity (recall) score for the forest tree model. The forest model had an almost imperceptibly higher precision, but looking at the f1 score the lower sensitivity (recall) outweighed the precision, and made for a less accurate model.